In [1]:
#NEW Data

In [2]:
import os
import cv2
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import save_model
from PIL import Image

In [20]:
# Load labels from the CSV file
labels_df = pd.read_csv('D:/School/Btech Project/archive/train.csv')
labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3662 entries, 0 to 3661
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id_code    3662 non-null   object
 1   diagnosis  3662 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 57.3+ KB


In [21]:
# Define image dimensions
image_height, image_width = 128, 128

In [22]:
# Initialize empty lists to store images and labels
images = []
labels = []

In [23]:
# Path to the folder containing all images
data_folder = 'D:/School/Btech Project/archive/colored_images/combined'

In [25]:
#Load images and labels from the folder and CSV file
for index, row in labels_df.iterrows():
    image_name = row['id_code']  # Image name from CSV without extension
    image_path = os.path.join(data_folder, f"{image_name}.png")  # Concatenate the file extension
    
    # Check if the image file exists
    if not os.path.exists(image_path):
        print(f"Image file not found: {image_path}")
        continue
    
    img = cv2.imread(image_path)
    img = cv2.resize(img, (image_width, image_height))
    images.append(img)
    labels.append(row['diagnosis'])


In [26]:
# Convert images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [27]:
print(f"Number of images: {len(images)}")
print(f"Number of labels: {len(labels)}")

Number of images: 7324
Number of labels: 7324


In [28]:
# Normalize images to [0, 1] range
images = images / 255.0

In [ ]:
# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
labels = to_categorical(labels, num_classes)

In [29]:
# Split data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

MemoryError: Unable to allocate 2.15 GiB for an array with shape (5859, 128, 128, 3) and data type float64

In [ ]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [15]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
22/22 [==============================] - 23s 466ms/step - loss: 1.5775 - accuracy: 0.2795 - val_loss: 1.5118 - val_accuracy: 0.3590
Epoch 2/10
22/22 [==============================] - 8s 383ms/step - loss: 1.4549 - accuracy: 0.3718 - val_loss: 1.3646 - val_accuracy: 0.3846
Epoch 3/10
22/22 [==============================] - 8s 373ms/step - loss: 1.3430 - accuracy: 0.4265 - val_loss: 1.2294 - val_accuracy: 0.4872
Epoch 4/10
22/22 [==============================] - 8s 375ms/step - loss: 1.2636 - accuracy: 0.4870 - val_loss: 1.2117 - val_accuracy: 0.5000
Epoch 5/10
22/22 [==============================] - 9s 414ms/step - loss: 1.2138 - accuracy: 0.5029 - val_loss: 1.2196 - val_accuracy: 0.5769
Epoch 6/10
22/22 [==============================] - 41s 2s/step - loss: 1.1704 - accuracy: 0.4986 - val_loss: 1.1764 - val_accuracy: 0.5128
Epoch 7/10
22/22 [==============================] - 10s 435ms/step - loss: 1.1677 - accuracy: 0.5086 - val_loss: 1.1370 - val_accuracy: 0.5256
Epoch 

In [17]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_accuracy}')

7/7 [==============================] - 0s 62ms/step - loss: 1.1930 - accuracy: 0.5285
Test accuracy: 0.5284973978996277


In [ ]:
# After training your model
model.save('D:/School/Btech Project/archive/colored_images/model/model.h5')